In [119]:
import pandas as pd

# ! options !
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


In [120]:
## Import Data ##

chemin_fichier_2018 = '../valeursfoncieres-2018.txt'
chemin_fichier_2019 = '../valeursfoncieres-2019.txt'
chemin_fichier_2020 = '../valeursfoncieres-2020.txt'
chemin_fichier_2021 = '../valeursfoncieres-2021.txt'


df_18 = pd.read_csv(chemin_fichier_2018, delimiter='|',nrows=200000)
df_19 = pd.read_csv(chemin_fichier_2019, delimiter='|',nrows=200000)
df_20 = pd.read_csv(chemin_fichier_2020, delimiter='|', nrows=200000)
df_21 = pd.read_csv(chemin_fichier_2021, delimiter='|',nrows=200000)

/tmp/ipykernel_8522/2316800614.py:9: DtypeWarning: Columns (24,26,28,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv(chemin_fichier_2018, delimiter='|',nrows=200000)
/tmp/ipykernel_8522/2316800614.py:10: DtypeWarning: Columns (24,26,28,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_19 = pd.read_csv(chemin_fichier_2019, delimiter='|',nrows=200000)
/tmp/ipykernel_8522/2316800614.py:11: DtypeWarning: Columns (23,24,26,28,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_20 = pd.read_csv(chemin_fichier_2020, delimiter='|', nrows=200000)
/tmp/ipykernel_8522/2316800614.py:12: DtypeWarning: Columns (23,24,26,28,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_21 = pd.read_csv(chemin_fichier_2021, delimiter='|',nrows=200000)


In [121]:
# Concat all years in one dataframe (and take some parts if we want)

df_18 = df_18.sample(frac=1, random_state=42)
df_19 = df_19.sample(frac=1, random_state=42)
df_20 = df_20.sample(frac=1, random_state=42)
df_21 = df_21.sample(frac=1, random_state=42)

df = pd.concat([df_18, df_19, df_20, df_21], axis=0)
#df = pd.concat([df_18], axis=0)


In [122]:
# Replace ' ' by '_'

df.columns = df.columns.str.replace(' ', '_')

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800000 entries, 119737 to 121958
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Identifiant_de_document     0 non-null       float64
 1   Reference_document          0 non-null       float64
 2   1_Articles_CGI              0 non-null       float64
 3   2_Articles_CGI              0 non-null       float64
 4   3_Articles_CGI              0 non-null       float64
 5   4_Articles_CGI              0 non-null       float64
 6   5_Articles_CGI              0 non-null       float64
 7   No_disposition              800000 non-null  int64  
 8   Date_mutation               800000 non-null  object 
 9   Nature_mutation             800000 non-null  object 
 10  Valeur_fonciere             793865 non-null  object 
 11  No_voie                     466133 non-null  float64
 12  B/T/Q                       30101 non-null   object 
 13  Type_de_v

In [124]:
# Affichez les pourcentages de NaN pour chaque colonne

pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)

Identifiant_de_document      100.00
Reference_document           100.00
1_Articles_CGI               100.00
2_Articles_CGI               100.00
3_Articles_CGI               100.00
4_Articles_CGI               100.00
5_Articles_CGI               100.00
No_disposition                 0.00
Date_mutation                  0.00
Nature_mutation                0.00
Valeur_fonciere                0.77
No_voie                       41.73
B/T/Q                         96.24
Type_de_voie                  45.38
Code_voie                      0.72
Voie                           0.72
Code_postal                    0.72
Commune                        0.00
Code_departement               0.00
Code_commune                   0.00
Prefixe_de_section            98.20
Section                        0.02
No_plan                        0.00
No_Volume                     99.79
1er_lot                       64.10
Surface_Carrez_du_1er_lot     89.03
2eme_lot                      91.36
Surface_Carrez_du_2eme_lot  

In [125]:
# Remove empty columns and keep only 'Vente' 

columns_to_remove = ["Identifiant_de_document", "Reference_document", "1_Articles_CGI", "2_Articles_CGI", "3_Articles_CGI", "4_Articles_CGI", "5_Articles_CGI",
                     "B/T/Q", "Prefixe_de_section", "No_Volume", "Type_local", "Identifiant_local", "Nature_culture_speciale",
                     "1er_lot", "Surface_Carrez_du_1er_lot", "2eme_lot", "Surface_Carrez_du_2eme_lot", "3eme_lot", "Surface_Carrez_du_3eme_lot", "4eme_lot",
                     "Surface_Carrez_du_4eme_lot", "5eme_lot", "Surface_Carrez_du_5eme_lot"]


df = df.drop(columns_to_remove, axis=1)
df = df[df['Nature_mutation'] == "Vente"] 
df = df.drop(columns=['Nature_mutation'])
    

pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)


No_disposition               0.00
Date_mutation                0.00
Valeur_fonciere              0.72
No_voie                     39.69
Type_de_voie                44.48
Code_voie                    0.03
Voie                         0.03
Code_postal                  0.04
Commune                      0.00
Code_departement             0.00
Code_commune                 0.00
Section                      0.01
No_plan                      0.00
Nombre_de_lots               0.00
Code_type_local             40.81
Surface_reelle_bati         40.89
Nombre_pieces_principales   40.89
Nature_culture              33.17
Surface_terrain             33.17
dtype: float64


In [126]:
# Create variable 'adresse' and remove variables use to build the adress

columns_for_adress = ["No_voie", "Type_de_voie", "Code_voie",
                      "Voie", "Code_commune", "Code_departement"]


df['Code_postal'] = df.groupby('Commune')['Code_postal'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))  # match les code postal vide avec les communes (bien)
    
df['No_voie'] = df['No_voie'].apply(lambda x: str(int(x)) if pd.notnull(
        x) else '').astype(str)  # change le type si non il y a une valeur sinon affiche rien

df['adresse'] = df['No_voie'] + ' ' + df['Type_de_voie'].astype(str) + ' ' + \
df['Code_voie'].astype(str) + ' ' + df['Voie'].astype(str) + ' ' + \
df['Code_postal'].astype(str) + ' ' + df['Commune']
    
    
# Après la création de l'adresse on remove les variables concatenées et les autres inutiles
df = df.drop(columns_for_adress, axis=1)


df.head()

,No_disposition,Date_mutation,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,adresse
119737,1,27/03/2018,"195000,00",6000.00,NICE,LZ,31,2,2.00,61.00,3.00,NaN,NaN,126 CHE 1725 DE LA COSTIERE 6000.0 NICE
72272,1,08/06/2018,"3000,00",3120.00,SAINT-CHRISTOPHE,D,552,0,NaN,NaN,NaN,P,1911.00,nan B018 CHAMP DES MOULINS 3120.0 SAINT-CHRIS...
158154,1,01/10/2018,"5220447,00",6400.00,CANNES,CL,152,0,4.00,NaN,NaN,BT,2653.00,1 BD 2780 DE L OBSERVATOIRE 6400.0 CANNES
65426,1,01/03/2018,"150000,00",3440.00,SAINT-HILAIRE,AC,67,0,2.00,43.00,3.00,S,231.00,2 RUE 0060 DE LA POSTE 3440.0 SAINT-HILAIRE
30074,1,20/02/2018,"210500,00",1400.00,CHANOZ-CHATENAY,B,386,0,NaN,NaN,NaN,J,315.00,nan B071 LE VILLAGE 1400.0 CHANOZ-CHATENAY


In [127]:
# Remove duplicates (date_mutation et adresse)

df = df.drop_duplicates(subset=["Date_mutation", "adresse"])

# Variable 'adresse' useless now => remove
df = df.drop("adresse", axis=1)

df.head()

,No_disposition,Date_mutation,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain
119737,1,27/03/2018,"195000,00",6000.00,NICE,LZ,31,2,2.00,61.00,3.00,NaN,NaN
72272,1,08/06/2018,"3000,00",3120.00,SAINT-CHRISTOPHE,D,552,0,NaN,NaN,NaN,P,1911.00
158154,1,01/10/2018,"5220447,00",6400.00,CANNES,CL,152,0,4.00,NaN,NaN,BT,2653.00
65426,1,01/03/2018,"150000,00",3440.00,SAINT-HILAIRE,AC,67,0,2.00,43.00,3.00,S,231.00
30074,1,20/02/2018,"210500,00",1400.00,CHANOZ-CHATENAY,B,386,0,NaN,NaN,NaN,J,315.00


In [128]:
# Affichez le nombre d'individus par modalité de la colonne 'Code_type_local'

comptage_modalites = df['Code_type_local'].value_counts()
print(comptage_modalites)


1.00    90303
2.00    86143
3.00    63700
4.00    14726
Name: Code_type_local, dtype: int64


In [129]:
print("before : ", len(df))

#df_modified = df_modified.dropna() # on ne fait plus ça

df['Valeur_fonciere'] = df['Valeur_fonciere'].astype(str).str.replace(',', '.').astype(float)  
    
print("after : ", len(df), "\n")

comptage_modalites = df['Code_type_local'].value_counts()
print(comptage_modalites)


before :  392760
after :  392760 

1.00    90303
2.00    86143
3.00    63700
4.00    14726
Name: Code_type_local, dtype: int64


In [130]:
masque_nan = df.isna()

# Utilisez sum() pour compter le nombre de valeurs manquantes (True) dans chaque colonne
nombre_nan_par_colonne = masque_nan.sum()
print(nombre_nan_par_colonne)

No_disposition                    0
Date_mutation                     0
Valeur_fonciere                1788
Code_postal                       0
Commune                           0
Section                          39
No_plan                           0
Nombre_de_lots                    0
Code_type_local              137888
Surface_reelle_bati          138269
Nombre_pieces_principales    138269
Nature_culture               147362
Surface_terrain              147362
dtype: int64


In [131]:
# Remove individus with at least one NaN and outliers

print("before : ", len(df))

# OUTLIERS 
df['Valeur_fonciere'] = df['Valeur_fonciere'].astype(str).str.replace(',', '.').astype(float)

df = df.dropna(subset=['Valeur_fonciere'])
Q1 = df['Valeur_fonciere'].quantile(0.25)
Q3 = df['Valeur_fonciere'].quantile(0.75)
    
#print(Q1)
#print(Q3)

IQR = Q3 - Q1

# Définissez les limites supérieure et inférieure pour détecter les outliers
limite_inferieure = Q1 - 0.3 * IQR
limite_superieure = Q3 + 1.7 * IQR

print(limite_inferieure)
print(limite_superieure,"\n")
    
df = df[(df['Valeur_fonciere'] >= limite_inferieure) & (df['Valeur_fonciere'] <= limite_superieure)]
    
print("after : ", len(df), "\n")

comptage_modalites = df['Code_type_local'].value_counts()
print(comptage_modalites)


before :  392760
-6200.0
545800.0 

after :  367819 

1.00    83087
2.00    82650
3.00    59876
4.00    12725
Name: Code_type_local, dtype: int64


In [132]:
# Créer la variable "day", "month" and "year" 

# NB : Utile si on la mets en 'int' et pas en objet, car le prix de l'immobilier augmente chaque année, donc plus l'année augmente plus le prix aussi
# alors qu'en 'objet' inutile car dans ce sont les data de 2022 qui vont être testés (à revoir quand même)


df['Date_mutation'] = pd.to_datetime(df['Date_mutation'], format='%d/%m/%Y')

# Extract the year component and create a new 'year' column
df['day'] = df['Date_mutation'].dt.day
df['day'] = df['day'].astype(object)

df['month'] = df['Date_mutation'].dt.month
df['month'] = df['month'].astype(object)

df['year'] = df['Date_mutation'].dt.year
df['year'] = df['year'].astype(object)

df = df.drop("Date_mutation", axis=1)
    

df.head()


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year
119737,1,195000.00,6000.00,NICE,LZ,31,2,2.00,61.00,3.00,NaN,NaN,27,3,2018
72272,1,3000.00,3120.00,SAINT-CHRISTOPHE,D,552,0,NaN,NaN,NaN,P,1911.00,8,6,2018
65426,1,150000.00,3440.00,SAINT-HILAIRE,AC,67,0,2.00,43.00,3.00,S,231.00,1,3,2018
30074,1,210500.00,1400.00,CHANOZ-CHATENAY,B,386,0,NaN,NaN,NaN,J,315.00,20,2,2018
23677,1,165000.00,1210.00,FERNEY-VOLTAIRE,AK,223,1,3.00,0.00,0.00,NaN,NaN,15,10,2018


In [133]:
# Drop 'Code_postal' NaN anc change type

df = df.dropna(subset=['Code_postal'])
df['Code_postal'] = df['Code_postal'].astype(int)
df['Code_postal'] = df['Code_postal'].astype(object)  # puis en objet

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 367819 entries, 119737 to 769
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   No_disposition             367819 non-null  int64  
 1   Valeur_fonciere            367819 non-null  float64
 2   Code_postal                367819 non-null  object 
 3   Commune                    367819 non-null  object 
 4   Section                    367784 non-null  object 
 5   No_plan                    367819 non-null  int64  
 6   Nombre_de_lots             367819 non-null  int64  
 7   Code_type_local            238338 non-null  float64
 8   Surface_reelle_bati        238093 non-null  float64
 9   Nombre_pieces_principales  238093 non-null  float64
 10  Nature_culture             227090 non-null  object 
 11  Surface_terrain            227090 non-null  float64
 12  day                        367819 non-null  object 
 13  month                      

,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year
119737,1,195000.00,6000,NICE,LZ,31,2,2.00,61.00,3.00,NaN,NaN,27,3,2018
72272,1,3000.00,3120,SAINT-CHRISTOPHE,D,552,0,NaN,NaN,NaN,P,1911.00,8,6,2018
65426,1,150000.00,3440,SAINT-HILAIRE,AC,67,0,2.00,43.00,3.00,S,231.00,1,3,2018
30074,1,210500.00,1400,CHANOZ-CHATENAY,B,386,0,NaN,NaN,NaN,J,315.00,20,2,2018
23677,1,165000.00,1210,FERNEY-VOLTAIRE,AK,223,1,3.00,0.00,0.00,NaN,NaN,15,10,2018


In [134]:
# Load longitude, latitude, 'Code_postal' dataframe

geo = pd.read_csv('../data/communes-departement-region.csv')

geo.head()


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.15,4.93,1.00,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.00,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.01,5.43,2.00,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.00,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.96,5.37,4.00,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.00,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,46.00,4.91,5.00,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.00,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.75,5.59,6.00,NaN,Ambléon,Ambléon,1,Ain,84.00,Auvergne-Rhône-Alpes


In [135]:
geo.rename(columns={'code_postal': 'Code_postal'}, inplace=True)
geo.rename(columns={'nom_commune_postal': 'Commune'}, inplace=True)
geo.rename(columns={'code_departement': 'Code_departement'}, inplace=True)

In [136]:
# voir les doublons

doublons = geo[geo.duplicated(subset=['Code_postal', 'Commune'], keep=False)]
# OK -> supprmime les doublons Code postal => Commune, car quelques fois il y a une 'petite' commune dedans la grande enfin bref
geo = geo.drop_duplicates(subset=['Code_postal', 'Commune'])


In [137]:

geo['Commune'] = geo['Commune'].str.replace('-', ' ') 
df['Commune'] = df['Commune'].str.replace('-', ' ')
geo['Commune'] = geo['Commune'].str.replace('ST', 'SAINT')  # format 'ST' et 'SAINT'
df['Commune'] = df['Commune'].str.replace('ST', 'SAINT')
geo['Commune'] = geo['Commune'].str.replace('\'', ' ')
df['Commune'] = df['Commune'].str.replace('\'', ' ')

geo_commune = geo[['Commune', 'Code_postal', 'latitude', 'longitude']]


# MERGE
df = pd.merge(df, geo_commune, on=['Commune', 'Code_postal'], how='left')  # OK -> merge la commune et le code postal, car desfois la commune est dans différent département 


# Le reste des communes non trouvé c'est des erreurs dans le fichier geo, exemple => LA SALLE DES ALPES, dans les data => "LA SALLE", IMPOSSIBLE À gérer ce genre d'erreur humaine => à enlever
print(df['longitude'].isna().sum()," individus avec un nom de commune pas trouvé, on les enlève du dataset")

# Drop not found
df = df.dropna(subset=['longitude'])

print("nombre final d'individus : ",len(df))

df.head()


7483  individus avec un nom de commune pas trouvé, on les enlève du dataset
nombre final d'individus :  360336


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,latitude,longitude
0,1,195000.00,6000,NICE,LZ,31,2,2.00,61.00,3.00,NaN,NaN,27,3,2018,43.71,7.24
1,1,3000.00,3120,SAINT CHRISAINTOPHE,D,552,0,NaN,NaN,NaN,P,1911.00,8,6,2018,46.17,3.58
2,1,150000.00,3440,SAINT HILAIRE,AC,67,0,2.00,43.00,3.00,S,231.00,1,3,2018,46.49,3.02
3,1,210500.00,1400,CHANOZ CHATENAY,B,386,0,NaN,NaN,NaN,J,315.00,20,2,2018,46.19,5.03
4,1,165000.00,1210,FERNEY VOLTAIRE,AK,223,1,3.00,0.00,0.00,NaN,NaN,15,10,2018,46.25,6.11


In [138]:
# Handle 'Corse' Département

def assign_department(code_postal):
    if 20000 <= code_postal <= 20190:
        return '2A' # Pour gérer la corse sud
    elif 20200 <= code_postal <= 20270:
        return '2B' # Pour gérer la corse nord
    elif len(str(code_postal)) == 4:
        dep = '0' + str(code_postal)[:1] # Pour gérer les 10 premiers département
        return dep
    else:
        dep = str(code_postal)[:2]
        return dep 


# Appliquez la fonction à la colonne 'Code_postal' en utilisant apply()
df['Departement'] = df['Code_postal'].apply(lambda x: assign_department(x))
df['Departement'] = df['Departement'].astype(str)

In [139]:
# Create a dataframe save with the Nan, before dropping them

df_with_nan = df.copy()

# exporter DF
df_with_nan.to_pickle('../data/df_with_nan.pkl')


In [140]:
# Drop the Nan remainings, for the classification model

training_classif_df = df.dropna()
training_classif_df['Code_type_local'] = training_classif_df['Code_type_local'].astype(
    int)

# exporter DF
training_classif_df.to_pickle('../data/training_classif_data.pkl')


/tmp/ipykernel_8522/2130254933.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_classif_df['Code_type_local'] = training_classif_df['Code_type_local'].astype(
